In [2]:
import pandas as pd
import numpy as np
import shapefile #pip install pyshp
from geopy.distance import vincenty
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import KMeans
import glob
import os

In [3]:
path = 'D:/Users/Qoo/Desktop/taipower/data/'

In [3]:
#匯入觀測站經緯度資料
obs_station = pd.read_csv(path + 'obs_station/obs_intro.csv',encoding='big5',usecols=[0,1,2,3,4,5,6])
old_names = list(obs_station.columns)
new_names = ['stno','stname','st_height','st_lon','st_lat','st_city','st_address']
obs_station.rename(columns=dict(zip(old_names,new_names)),inplace=True)
obs_station.stno = obs_station.stno.str.encode('big5')

In [5]:
#匯入觀測站颱風雨量資料
all_files = glob.glob(os.path.join(path+'typhoon_rain/','*.csv'))
for file in all_files:
    try:
        pd.read_csv(file,encoding='big5',usecols=[0,1,2,3])
    except:
        print 'error at %s' %file

tp_rain = pd.concat(pd.read_csv(file,encoding='big5') for file in all_files )
tp_rain['stno'] = tp_rain['stno'].astype('str')
tp_rain['typhoon'] =  tp_rain['typhoon_name'].str[4:]
tp_rain.accu_end_time = pd.to_datetime(tp_rain.accu_end_time)
tp_rain['day'] = tp_rain.accu_end_time.dt.day
#tp_rain2 = pd.merge(tp_rain,obs_station,how='left',on='stno')
tp_rain.loc[tp_rain.typhoon =='MERANTI','typhoon'] = 'MERANTIANDMALAKAS'
tp_rain.loc[tp_rain.typhoon =='MALAKAS','typhoon'] = 'MERANTIANDMALAKAS'
tp_rain.loc[tp_rain.typhoon =='NESAT','typhoon'] = 'NESATANDHAITANG'
tp_rain.loc[tp_rain.typhoon =='HAITANG','typhoon'] = 'NESATANDHAITANG'
print len(tp_rain)
#print len(tp_rain2)

#檢查是否有缺漏的站
#key = tp_rain2.stname.isnull()
#tp_rain2[key].stno.value_counts()

6993


In [6]:
tp_rain.to_csv(path +'tp_rain.csv',encoding='big5')

In [7]:
#讀取村里邊界經緯度
def read_shapefile(shp_path):
	"""
	Read a shapefile into a Pandas dataframe with a 'coords' column holding
	the geometry information. This uses the pyshp package
	"""
	import shapefile

	#read file, parse out the records and shapes
	sf = shapefile.Reader(shp_path)
	fields = [x[0] for x in sf.fields][1:]
	records = sf.records()
	shps = [s.points for s in sf.shapes()]

	#write into a dataframe
	df = pd.DataFrame(columns=fields, data=records)
	df = df.assign(coords=shps)

	return df

vil_latlon = read_shapefile(path + "village_latlon/VILLAGE_MOI_1060831.shp")

#計算城鎮中心點經緯度
for element in range(0,len(vil_latlon)):
    vil_latlon.loc[element,'vil_lon'] = pd.DataFrame(vil_latlon.coords[element]).mean()[0]
    vil_latlon.loc[element,'vil_lat'] = pd.DataFrame(vil_latlon.coords[element]).mean()[1]
    
vil_latlon = vil_latlon.iloc[:,[0,1,2,3,11,12]]
vil_latlon.loc[0,['VILLCODE','VILLNAME']] = ['10013030023','大鵬里']
vil_latlon = vil_latlon.loc[vil_latlon.VILLNAME !='',:]
vil_latlon = vil_latlon.reset_index(drop=True)
vil_latlon.to_csv(path+'vil_raw_intro.csv')

In [8]:
#匯入先前已整理好之資料
df = pd.read_csv(path + 'df.csv')

In [9]:
#為每個村里找出所屬的(最近的)觀測站
obs_station.loc[:,'st_lonlat'] = obs_station.loc[:,['st_lon','st_lat']].apply(tuple,axis=1)
vil_latlon.loc[:,'vil_lonlat'] = vil_latlon.loc[:,['vil_lon','vil_lat']].apply(tuple,axis=1)

#刪除用不到之測站
obs_station = obs_station[0:33]

#回傳最短距離測站的index
def closest(point, points):
    distance = [vincenty(point, i).miles for i in points]
    return distance.index(min(distance))
'''
point = (121,25)
points = [(124,42),(132,22),(120,55)]
distance = [vincenty(point, i).miles for i in points]
distance.index(min(distance))
'''
for i in range(0,len(vil_latlon)):
    vil_latlon.loc[i,'stno_index'] = closest(vil_latlon.loc[i,'vil_lonlat'], obs_station.loc[:,'st_lonlat'])
obs_station.loc[:,'stno_index'] = obs_station.index
vil_latlon2 = pd.merge(vil_latlon, obs_station, how='left', on='stno_index')
vil_latlon2 = vil_latlon2.iloc[:,[0,1,2,3,4,5,8,9,10,11,12,13]]
vil_latlon2.to_csv(path +'village_intro.csv',encoding='utf-8')

#檢查是否有無對應的地區與測站
key = vil_latlon2.stno.isnull()
vil_latlon2[key]

,VILLCODE,COUNTYNAME,TOWNNAME,VILLNAME,vil_lon,vil_lat,stno,stname,st_height,st_lon,st_lat,st_city


In [11]:
#將訓練資料與地區對應的測站合併
for i in range(0,len(vil_latlon2)):
    if vil_latlon2.loc[i,'VILLCODE'][3:5] =='00':
        vil_latlon2.loc[i,'VilCode'] = vil_latlon2.loc[i,'VILLCODE'][0:3] +  vil_latlon2.loc[i,'VILLCODE'][5:7] + '00-'+  vil_latlon2.loc[i,'VILLCODE'][8:11]
    else:
        vil_latlon2.loc[i,'VilCode'] = vil_latlon2.loc[i,'VILLCODE'][0:7] +  '-'+  vil_latlon2.loc[i,'VILLCODE'][8:11]


In [12]:
df2 = pd.merge(df,vil_latlon2,how='left', on='VilCode')

In [13]:
df2.to_csv(path +'df2.csv',encoding='utf-8')

In [14]:
#找出沒有對應到的區域
key = df2.stno.isnull()
miss = df2[key]
miss.groupby(['CityName','TownName','VilName']).size().reset_index()

,CityName,TownName,VilName,0


In [16]:
# df 多出 大鵬里
print set(df.VilCode) - set(vil_latlon2.VilCode)
#df.loc[df.VilCode=='1001303-023',:]

set([])


In [17]:
# vil_latlon 多出 中崙里
print set(vil_latlon2.VilCode) - set(df.VilCode)
vil_latlon2.loc[vil_latlon2.VilCode =='1000401-031',:]

set(['1000401-031'])


,VILLCODE,COUNTYNAME,TOWNNAME,VILLNAME,vil_lon,vil_lat,stno,stname,st_height,st_lon,st_lat,st_city,VilCode
7823,10004010031,新竹縣,竹北市,中崙里,121.003926,24.821014,467571,新竹,26.9,121.0142,24.8279,新竹縣,1000401-031


In [18]:
#檢查是否每一個測站都有記錄到雨量
#兩處沒有記錄到降雨
print len(np.unique(tp_rain.stno))
print len(np.unique(df2.stno))
print set(df2.stno) - set(tp_rain.stno)
print set(tp_rain.stno) - set(df2.stno)
obs_station.loc[(obs_station.stno =='467790')|(obs_station.stno =='466850'),:]

31
33
set(['467790', '466850'])
set([])


,stno,stname,st_height,st_lon,st_lat,st_city,st_address,st_lonlat,stno_index
0,466850,五分山雷達站,756.0,121.7812,25.0712,新北市,瑞芳區靜安路四段1巷1號,"(121.7812, 25.0712)",0
30,467790,墾丁雷達站,42.0,120.8080,21.9482,屏東縣,恆春鎮燈塔路51巷33號,"(120.808, 21.9482)",30


In [19]:
#不同颱風在各測站下的平均每小時累積雨量
mean_hour_rain = tp_rain.groupby(['typhoon','stno'], as_index=False).agg({'accu_value':'mean'})
mean_hour_rain.columns.values[2] = 'mean_accu_hour_rain'

#不同颱風在各測站下的平均每日累積雨量
mean_day_rain = tp_rain.groupby(['day','typhoon','stno'], as_index=False).agg({'accu_value':'sum'})
mean_day_rain = mean_day_rain.groupby(['typhoon','stno'], as_index=False).agg({'accu_value':'mean'})
mean_day_rain.columns.values[2] = 'mean_accu_day_rain'

#不同颱風在各測站下總累積雨量
total_rain = tp_rain.groupby(['typhoon','stno'], as_index=False).agg({'accu_value':'sum'})
total_rain.columns.values[2] = 'accu_rain'

#達到幾次大雨指標(每日累積降雨 80MM以上)
heavy_rain = tp_rain.groupby(['day','typhoon','stno'], as_index=False).agg({'accu_value':'sum'})
heavy_rain_times = heavy_rain.loc[heavy_rain.accu_value >= 80,:].groupby(['typhoon','stno'], 
                as_index=False).agg({'accu_value':'count'})
heavy_rain_times.columns.values[2] = 'heavy_rain_count_rule1'

#達到幾次大雨指標(每小時累積降雨 40MM以上)
heavy_rain_times2 = tp_rain.loc[tp_rain.accu_value >=40,:].groupby(['typhoon','stno'],as_index=False).agg({'accu_value':'count'})
heavy_rain_times2.columns.values[2] = 'heavy_rain_count_rule2'

#達到幾次豪雨指標(每日累積降雨 200MM以上)
how_rain = tp_rain.groupby(['day','typhoon','stno'], as_index=False).agg({'accu_value':'sum'})
how_rain_times1 = how_rain.loc[how_rain.accu_value >= 200,:].groupby(['typhoon','stno'], 
                as_index=False).agg({'accu_value':'count'})
how_rain_times1.columns.values[2] = 'how_rain_count_rule1'

#達到幾次豪雨指標(三小時累積降雨 100MM以上)


#達到幾次大豪雨指標(每日累積降雨 350MM以上)
big_how_rain = tp_rain.groupby(['day','typhoon','stno'], as_index=False).agg({'accu_value':'sum'})
big_how_rain_times = big_how_rain.loc[big_how_rain.accu_value >= 350,:].groupby(['typhoon','stno'], 
                as_index=False).agg({'accu_value':'count'})
big_how_rain_times.columns.values[2] = 'big_how_rain_count'

#達到幾次超大豪雨指標(每日累積降雨 500MM以上)
big_big_how_rain = tp_rain.groupby(['day','typhoon','stno'], as_index=False).agg({'accu_value':'sum'})
big_big_how_rain_times = big_big_how_rain.loc[big_big_how_rain.accu_value >= 500,:].groupby(['typhoon','stno'], 
                as_index=False).agg({'accu_value':'count'})
big_big_how_rain_times.columns.values[2] = 'big_big_how_rain_count'

#不同颱風在各測站下的每小時最大累積雨量
max_hour_rain = tp_rain.groupby(['typhoon','stno'], as_index=False).agg({'accu_value':'max'})
max_hour_rain.columns.values[2] = 'max_hour_accu_rain'
#不同颱風在各測站下的每日最大累積雨量
max_day_rain = tp_rain.groupby(['day','typhoon','stno'], as_index=False).agg({'accu_value':'sum'}).groupby(['typhoon','stno'], as_index=False).agg({'accu_value':'max'})
max_day_rain.columns.values[2] = 'max_day_accu_rain'

In [20]:
df2 = pd.merge(df2, mean_hour_rain, on=['stno','typhoon'],how ='left')
df2 = pd.merge(df2, mean_day_rain, on=['stno','typhoon'],how ='left')
df2 = pd.merge(df2, total_rain, on=['stno','typhoon'],how ='left')
df2 = pd.merge(df2, heavy_rain_times, on=['stno','typhoon'],how ='left')
df2 = pd.merge(df2, heavy_rain_times2, on=['stno','typhoon'],how ='left')
df2 = pd.merge(df2, how_rain_times1, on=['stno','typhoon'],how ='left')
df2 = pd.merge(df2, big_how_rain_times, on=['stno','typhoon'],how ='left')
df2 = pd.merge(df2, big_big_how_rain_times, on=['stno','typhoon'],how ='left')
df2 = pd.merge(df2, max_hour_rain, on=['stno','typhoon'],how ='left')
df2 = pd.merge(df2, max_day_rain, on=['stno','typhoon'],how ='left')

In [23]:
#特別處理雙颱資料
tp_rain.loc[(tp_rain.typhoon =='MERANTI') & (tp_rain.typhoon =='MALAKAS')]

,accu_end_time,accu_value,num,stno,stntype,typhoon_cht_name,typhoon_name,typhoon,day


In [25]:
#補植 1.兩處沒有記錄到降雨的觀測站 以及 沒有發生大雨警報的資料欄位
df3 = pd.concat([ df2.iloc[:,0:55], df2.iloc[:,55:65].fillna(0)],axis=1)
print len(df3.columns.values)

65


In [26]:
df3.to_csv(path +'df3.csv',encoding='utf-8',index=False)

In [27]:
#匯入颱風風力觀測站資料
all_files = glob.glob(os.path.join(path+'typhoon_wind/','*.csv'))
for file in all_files:
    try:
        pd.read_csv(file,encoding='big5',usecols=[0,1,2,3])
    except:
        print 'error at %s' %file

tp_wind = pd.concat(pd.read_csv(file,encoding='big5') for file in all_files )

In [29]:
tp_wind['stno'] = tp_wind['stno'].astype('str')
tp_wind['typhoon'] = tp_wind.typhoon_name.str[4:]
tp_wind.ObsTime = pd.to_datetime(tp_wind.ObsTime)
tp_wind['day'] = tp_wind.ObsTime.dt.day
tp_wind.loc[tp_wind.typhoon =='MERANTI','typhoon'] = 'MERANTIANDMALAKAS'
tp_wind.loc[tp_wind.typhoon =='MALAKAS','typhoon'] = 'MERANTIANDMALAKAS'
tp_wind.loc[tp_wind.typhoon =='NESAT','typhoon'] = 'NESATANDHAITANG'
tp_wind.loc[tp_wind.typhoon =='HAITANG','typhoon'] = 'NESATANDHAITANG'
#刪除極端值(風速、風向為負值)
#tp_wind.loc[tp_wind.WDMax <0 ,'WDMax'] = 0
#tp_wind.loc[tp_wind.WDGust <0 ,'WDGust'] = 0
tp_wind.loc[tp_wind.WSGust <0 ,'WSGust'] = 0
tp_wind.loc[tp_wind.WSMax <0 ,'WSMax'] = 0

In [30]:
tp_wind.to_csv(path+'tp_wind.csv', encoding='utf-8', index=False)

In [32]:
#WDMax方向分群
tp_wind.loc[(tp_wind.WDMax == 0) | (tp_wind.WDMax == 360),'wind_direction_max'] = 1
tp_wind.loc[(tp_wind.WDMax > 0) & (tp_wind.WDMax < 45),'wind_direction_max'] = 2
tp_wind.loc[tp_wind.WDMax == 45,'wind_direction_max'] = 3
tp_wind.loc[(tp_wind.WDMax > 45) & (tp_wind.WDMax < 90),'wind_direction_max'] = 4
tp_wind.loc[tp_wind.WDMax == 90,'wind_direction_max'] = 5
tp_wind.loc[(tp_wind.WDMax > 90) & (tp_wind.WDMax < 135),'wind_direction_max'] = 6
tp_wind.loc[tp_wind.WDMax == 135,'wind_direction_max'] = 7
tp_wind.loc[(tp_wind.WDMax > 135) & (tp_wind.WDMax < 180),'wind_direction_max'] = 8
tp_wind.loc[tp_wind.WDMax == 180,'wind_direction_max'] = 9
tp_wind.loc[(tp_wind.WDMax > 180) & (tp_wind.WDMax < 225),'wind_direction_max'] = 10
tp_wind.loc[tp_wind.WDMax == 225,'wind_direction_max'] = 11
tp_wind.loc[(tp_wind.WDMax > 225) & (tp_wind.WDMax < 270),'wind_direction_max'] = 12
tp_wind.loc[tp_wind.WDMax == 270,'wind_direction_max'] = 13
tp_wind.loc[(tp_wind.WDMax > 270) & (tp_wind.WDMax < 315),'wind_direction_max'] = 14
tp_wind.loc[tp_wind.WDMax == 315,'wind_direction_max'] = 15
tp_wind.loc[(tp_wind.WDMax > 315) & (tp_wind.WDMax < 360),'wind_direction_max'] = 16
#WDGust方向分群
tp_wind.loc[(tp_wind.WDGust == 0) | (tp_wind.WDGust == 360),'wind_direction_gust'] = 1
tp_wind.loc[(tp_wind.WDGust > 0) & (tp_wind.WDGust < 45),'wind_direction_gust'] = 2
tp_wind.loc[tp_wind.WDGust == 45,'wind_direction_gust'] = 3
tp_wind.loc[(tp_wind.WDGust > 45) & (tp_wind.WDGust < 90),'wind_direction_gust'] = 4
tp_wind.loc[tp_wind.WDGust == 90,'wind_direction_gust'] = 5
tp_wind.loc[(tp_wind.WDGust > 90) & (tp_wind.WDGust < 135),'wind_direction_gust'] = 6
tp_wind.loc[tp_wind.WDGust == 135,'wind_direction_gust'] = 7
tp_wind.loc[(tp_wind.WDGust > 135) & (tp_wind.WDGust < 180),'wind_direction_gust'] = 8
tp_wind.loc[tp_wind.WDGust == 180,'wind_direction_gust'] = 9
tp_wind.loc[(tp_wind.WDGust > 180) & (tp_wind.WDGust < 225),'wind_direction_gust'] = 10
tp_wind.loc[tp_wind.WDGust == 225,'wind_direction_gust'] = 11
tp_wind.loc[(tp_wind.WDGust > 225) & (tp_wind.WDGust < 270),'wind_direction_gust'] = 12
tp_wind.loc[tp_wind.WDGust == 270,'wind_direction_gust'] = 13
tp_wind.loc[(tp_wind.WDGust > 270) & (tp_wind.WDGust < 315),'wind_direction_gust'] = 14
tp_wind.loc[tp_wind.WDGust == 315,'wind_direction_gust'] = 15
tp_wind.loc[(tp_wind.WDGust > 315) & (tp_wind.WDGust < 360),'wind_direction_gust'] = 16

In [33]:
tp_wind['WSD_M'] = tp_wind.WSMax *tp_wind.wind_direction_max
tp_wind['WSD_G'] = tp_wind.WSGust * tp_wind.wind_direction_gust

In [34]:
#平均每小時最大風速
mean_hr_wsm = tp_wind.groupby(['typhoon','stno'], as_index=False).agg({'WSMax':'mean'})
mean_hr_wsm.columns.values[2] = 'mean_hr_wsmax'
#平均每小時瞬間極大風速
mean_hr_wsg = tp_wind.groupby(['typhoon','stno'], as_index=False).agg({'WSGust':'mean'})
mean_hr_wsg.columns.values[2] = 'mean_hr_wsgust'
#最大小時風速
max_hr_wsm = tp_wind.groupby(['typhoon','stno'], as_index=False).agg({'WSMax':'max'})
max_hr_wsm.columns.values[2] = 'max_hr_wsmax'
#最大小時瞬間極大風速
max_hr_wsg = tp_wind.groupby(['typhoon','stno'], as_index=False).agg({'WSGust':'max'})
max_hr_wsg.columns.values[2] = 'max_hr_wsgust'
#測站測得颱風最大風速每小時達到烈風以上的總次數
total_daily_sum_WSM = tp_wind.loc[tp_wind.WSMax >20.8,:].groupby(['typhoon','stno'], as_index=False).agg({'WSMax':'count'})
total_daily_sum_WSM.columns.values[2] = 'total_daily_sum_WSMax'
#測站測得颱風瞬間極大風速每小時達到烈風以上的總次數
total_daily_sum_WSG =tp_wind.loc[tp_wind.WSGust >20.8,:].groupby(['typhoon','stno'], as_index=False).agg({'WSMax':'count'})
total_daily_sum_WSG.columns.values[2] = 'total_daily_sum_WSGust'
#觀測站每日平均風速達烈風以上的日數
#最大風各風向總吹拂小時次數
wind_direction_m = tp_wind.groupby(['typhoon','stno'], as_index=False).agg({'wind_direction_max':'count'})
wind_direction_m.columns.values[2] = 'total_wind_direction_m'
#極大風各風向總吹拂小時次數
wind_direction_g = tp_wind.groupby(['typhoon','stno'], as_index=False).agg({'wind_direction_gust':'count'})
wind_direction_g.columns.values[2] = 'total_wind_direction_g'
#平均風向
mean_wdm = tp_wind.groupby(['typhoon','stno'], as_index=False).agg({'WDMax':'mean'})
mean_wdm.columns.values[2] = 'mean_wdmax'
mean_wdg = tp_wind.groupby(['typhoon','stno'], as_index=False).agg({'WDGust':'mean'})
mean_wdg.columns.values[2] = 'mean_wdgust'
#交互作用
sd1 = tp_wind.groupby(['typhoon','stno'], as_index=False).agg({'WSD_M':'mean'})
sd1.columns.values[2] = 'mean_WSDmax'
sd2 = tp_wind.groupby(['typhoon','stno'], as_index=False).agg({'WSD_M':'max'})
sd2.columns.values[2] = 'max_WSDmax'
sd3 = tp_wind.groupby(['typhoon','stno'], as_index=False).agg({'WSD_G':'mean'})
sd3.columns.values[2] = 'mean_WSDgust'
sd4 = tp_wind.groupby(['typhoon','stno'], as_index=False).agg({'WSD_G':'mean'})
sd4.columns.values[2] = 'max_WSDgust'

In [35]:
df4 = pd.merge(df3, mean_hr_wsm, on=['stno','typhoon'],how ='left')
df4 = pd.merge(df4, mean_hr_wsg, on=['stno','typhoon'],how ='left')
df4 = pd.merge(df4, max_hr_wsm, on=['stno','typhoon'],how ='left')
df4 = pd.merge(df4, max_hr_wsg, on=['stno','typhoon'],how ='left')
df4 = pd.merge(df4, total_daily_sum_WSM, on=['stno','typhoon'],how ='left')
df4 = pd.merge(df4, total_daily_sum_WSG, on=['stno','typhoon'],how ='left')
df4 = pd.merge(df4, wind_direction_m, on=['stno','typhoon'],how ='left')
df4 = pd.merge(df4, wind_direction_g, on=['stno','typhoon'],how ='left')
df4 = pd.merge(df4, mean_wdm, on=['stno','typhoon'],how ='left')
df4 = pd.merge(df4, mean_wdg, on=['stno','typhoon'],how ='left')
df4 = pd.merge(df4, sd1, on=['stno','typhoon'],how ='left')
df4 = pd.merge(df4, sd2, on=['stno','typhoon'],how ='left')
df4 = pd.merge(df4, sd3, on=['stno','typhoon'],how ='left')
df4 = pd.merge(df4, sd4, on=['stno','typhoon'],how ='left')

In [36]:
City_danger_river_num = pd.read_csv(path +'City_danger_river_num.csv')
#City_danger_river_num.CityName = City_danger_river_num.CityName.str.encode=('utf-8')
#print type(df5.CityName[1])
#print type(City_danger_river_num.CityName[1])
df4 = pd.merge(df4, City_danger_river_num, how='left', on='CityName')

In [37]:
#補值
df5 = pd.concat([ df4.iloc[:,0:65], df4.iloc[:,65:].fillna(0)],axis=1)

In [39]:
#地區分組
kmeans = KMeans(n_clusters=20, random_state=2017).fit(df5[['vil_lon','vil_lat']])
df5['region_cluster'] = kmeans.predict(df5[['vil_lon','vil_lat']])

In [40]:
#颱風土石流警報
landslide = pd.read_csv(path+'landslide_alert.csv' )
landslide
df5 = pd.merge(df5, landslide, how='left', on=['typhoon','year'])

In [41]:
df5.to_csv(path +'df4.csv',encoding='utf-8',index=False)